# (R) 비트코인의 daily/hourly data를 바탕으로

- 거품 측정 (AR(1)의 alpha > 1인지 확인) 및 비트코인이 일반 금융자산이 거품상태일 때와 유사한 양상을 보이는지 확인. 
- GJR-GARCH(1,1)의 gamma 확인 
- 거래량 데이터를 이용해 ARMAX/GARCHX 모델 수립
- daily/hourly data에 따라 차이가 있는지 확인


## 0. Initial settings. 

In [68]:
Sys.setlocale('LC_ALL', 'english')

r = getOption('repos')
r['CRAN'] <- 'http://cran.seoul.go.kr' # 서울시 빅데이터 캠퍼스
options(repos=r)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] "LC_COLLATE=English_United States.1252;LC_CTYPE=English_United States.1252;LC_MONETARY=English_United States.1252;LC_NUMERIC=C;LC_TIME=English_United States.1252"

In [88]:
require(forecast)
require(zoo)
require(fGarch)
require(rugarch)
require(psych)
require(aTSA)
require(fBasics)
require(tidyverse)
require(dplyr)

# 1. Import data and convert to zoo 

In [99]:
binance_d <- read.table("./mydata/Binance_BTCUSDT_d.csv", skip=1, sep=',', header=TRUE)
binance_h <- read.table("./mydata/Binance_BTCUSDT_1h.csv", skip=1, sep=',', header=TRUE)

In [100]:
binance_d <- select(binance_d, -c(Symbol))
binance_h <- select(binance_h, -c(Symbol))

In [101]:
binance_d <- read.zoo(binance_d, format='%Y-%m-%d')
head(binance_d)

              Open    High     Low   Close Volume.BTC Volume.USDT
2017-08-17 4469.93 4485.39 4200.74 4285.08     647.86     2812379
2017-08-18 4285.08 4371.52 3938.77 4108.37    1178.07     4994494
2017-08-19 4108.37 4184.69 3850.00 4139.98     371.15     1508239
2017-08-20 4139.98 4211.08 4032.62 4086.29     463.54     1915636
2017-08-21 4086.29 4119.62 3911.79 4016.00     685.12     2770592
2017-08-22 4016.00 4104.82 3400.00 4040.00     939.91     3648727

In [102]:
# binance_h <- read.zoo(binance_h, format='%Y-%m-%d %I-%p') # locale을 korea에서 us로 바꿔주니 해결. 
# 하지만 위의 코드는 이상하게 hour을 지워버림. 

binance_h$Date <- ymd_h(binance_h$Date) # lubridate로 대체
binance_h <- read.zoo(binance_h) 
head(binance_h)

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

                        Open     High     Low   Close Volume.BTC Volume.USDT
2017-08-17 04:00:00 16199.91 16199.91 4261.32 4308.83    44.5100   190952.85
2017-08-17 05:00:00  4308.83  4328.69 4291.37 4315.32    23.2300   100304.82
2017-08-17 06:00:00  4315.32  4345.45 4309.37 4324.35     7.2300    31282.31
2017-08-17 07:00:00  4324.35  4349.99 4287.41 4349.99     4.4400    19241.06
2017-08-17 08:00:00  4349.99  4377.85 4333.32 4360.69     0.9499     4139.70
2017-08-17 09:00:00  4360.69  4445.78 4360.00 4444.00    10.7600    47219.36

In [103]:
typeof(binance_d)

[1] "double"

In [104]:
typeof(binance_h)

[1] "double"

In [113]:
binance_d_before <- binance_d[index(binance_d) < as.Date("2018-10-01")]
binance_d_after <- binance_d[index(binance_d) >= as.Date("2018-10-01")]

binance_h_before <- binance_h[index(binance_h) < as.Date("2018-10-01")]
binance_h_after <- binance_h[index(binance_h) >= as.Date("2018-10-01")]